# Tutorial for simulated data
This tutorial provides step-by-step instructions for reproducing the simulation results presented in our paper. For each simulated dataset, we generated two modalities: one representing the transcriptome and the other representing the proteome. These datasets include multiple ground truth patterns—some shared between both modalities and others unique to a single modality. The simulated data are then used as input for our algorithm, which produces two types of outputs based on user preference.

In [4]:
# SpaMV results
import anndata
from sklearn.metrics import adjusted_rand_score

from SpaMV.spamv import SpaMV
from SpaMV.utils import clr_normalize_each_cell, clustering
import scanpy as sc
import matplotlib.pyplot as plt
import wandb

for dataset in ['1_Simulation', '2_Simulation', '3_Simulation']:
    data_rna = sc.read_h5ad('Data/' + dataset + '/adata_RNA.h5ad')
    data_pro = sc.read_h5ad('Data/' + dataset + '/adata_ADT.h5ad')
    omics_names = ['Omics 1', 'Omics 2']
    sc.pp.normalize_total(data_rna)
    sc.pp.log1p(data_rna)
    sc.pp.pca(data_rna, n_comps=30)
    data_rna = anndata.AnnData(data_rna.obsm['X_pca'], obs=data_rna.obs, obsm=data_rna.obsm)
    data_pro = clr_normalize_each_cell(data_pro)
    sc.pp.pca(data_pro, n_comps=30)
    data_pro = anndata.AnnData(data_pro.obsm['X_pca'], obs=data_pro.obs, obsm=data_pro.obsm)

    # wandb.init(project=dataset)
    # wandb.login()
    # model = SpaMV([data_rna, data_pro], interpretable=False, omics_names=omics_names, max_epochs=400)
    # model.train(dataset)
    # wandb.finish()

    model = SpaMV([data_rna, data_pro], interpretable=False, omics_names=omics_names, max_epochs_stage1=400)
    model.train()

    data_rna.obsm['SpaMV'] = model.get_embedding()
    # data_rna = clustering(data_rna, n_clusters=10, key='SpaMV' ,use_pca=True)
    clustering(data_rna, n_clusters=10, key='SpaMV' ,use_pca=True)

    fig, axes = plt.subplots(1, 2, figsize=(8, 4))  # 1 row, 2 columns

    sc.pp.neighbors(data_rna, use_rep='SpaMV')
    sc.tl.umap(data_rna)
    sc.pl.umap(data_rna, color='SpaMV', ax=axes[0], show=False, legend_loc='none', s=20, title='UMAP')
    sc.pl.embedding(data_rna, color='SpaMV', basis='spatial', s=200, show=False, title='SpaMV on {}\nARI: {:.3f}'.format(dataset, adjusted_rand_score(data_rna.obs['cluster'], data_rna.obs['SpaMV'])), ax=axes[1])
    plt.tight_layout()
    plt.show()

cpu
Loading data


Epoch Loss:212.442:  19%|█▉        | 75/400 [01:08<04:55,  1.10it/s]


KeyboardInterrupt: 

# Interpretable dimension reduction

In [ ]:
import numpy as np

from SpaMV_copy.spamv import SpaMV
from SpaMV.utils import clr_normalize_each_cell, plot_embedding_results
import scanpy as sc
import wandb

omics_names = ['Transcriptomics', 'Proteomics']
for dataset in ['1_Simulation', '2_Simulation', '3_Simulation']:
    # for dataset in ['3_Simulation_o']:
    print('dataset:', dataset)
    data_rna = sc.read_h5ad('Data/' + dataset + '/adata_RNA.h5ad')
    sc.pp.normalize_total(data_rna)
    sc.pp.log1p(data_rna)
    data_pro = sc.read_h5ad('Data/' + dataset + '/adata_ADT.h5ad')
    data_pro = clr_normalize_each_cell(data_pro)

    wandb.init(project=dataset)
    wandb.login()
    model = SpaMV([data_rna, data_pro], zs_dim=10, zp_dims=[10, 10], betas=[3, 3], interpretable=True, omics_names=omics_names, threshold_background=5, max_epochs_stage2=200)
    model.train(dataset)
    wandb.finish()

    z, w = model.get_embedding_and_feature_by_topic()
    # Define the softmax function
    def softmax(x):
        e_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
        return e_x / e_x.sum()
    z = z.apply(lambda row: softmax(row), axis=1)
    plot_embedding_results([data_rna, data_pro], omics_names, z, w, save=False, show=True, size=350)

/opt/anaconda3/envs/SpaMV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/SpaMV/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/opt/anaconda3/envs/SpaMV/lib/python3.12/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, ge